## 01.09.2025
### Task description for Emily: creating master dataframe for BOLD functional connectivity analysis

In [15]:
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import input_data, datasets
import os
import pickle

In [16]:
## If this is set to true, then the notebook will attempt to generate a giant data structure containing
## all the time series for all the subjects and conditions and voxels. This is a very large data structure
generate_time_series = False

# data_root is only used if generate_time_series is set to True
data_root = '/Users/jacekdmochowski/PROJECTS/fus/data/resampled_bold_flywheel/'

# if you change these, you straight trippin
idx_pre = range(60,300) # we start at 60 instead of 0 because of some strange artifact at the beginning of the time series
idx_fus = range(300,600)
idx_post = range(600,900)

In [17]:
# load DiFuMo atlas
difumo = datasets.fetch_atlas_difumo(dimension=1024)
labels = difumo.labels  # List of 1024 anatomical labels
atlas_img = nib.load(difumo.maps)
print(labels[:3])

[(1, 'Retrocalcarine cortex RH', 'VisCent', 'VisCent', 0.44814054, 0.53647558, 1.53973010e-02)
 (2, 'Superior longitudinal fasciculus II mid-posterior RH', 'No network found', 'No network found', 0.01732641, 0.98266104, 2.36605469e-05)
 (3, 'Arcuate fasciculus mid-anterior RH', 'DorsAttnB', 'ContA', 0.50291488, 0.47371745, 2.33761400e-02)]


In [18]:
def get_folders(path):
    """Gets all folders within a specified path."""
    folders = []
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(entry.name)
    return folders

In [19]:
def load_and_prepare_data(bold_files, confounds_files, difumo_atlas):
    """
    Load and prepare BOLD data using the DiFuMo probabilistic atlas

    Parameters:
    -----------
    bold_files : list of str
        Paths to preprocessed BOLD data for each subject/session
    confounds_files : list of str
        Paths to confound regressors from fMRIprep
    difumo_atlas : str or NiftiImage
        Path to DiFuMo probabilistic atlas or loaded atlas image
    target_roi_indices : list of int
        Indices of ROIs in target region (subgenual ACC)

    Returns:
    --------
    time_series_dict : dict
        Dictionary containing cleaned time series for each subject/condition
    """
    # Use NiftiMapsMasker instead of NiftiLabelsMasker for probabilistic atlas
    masker = input_data.NiftiMapsMasker(
        maps_img=difumo_atlas,
        standardize=True,
        detrend=False,
        low_pass=0.1,
        high_pass=0.01,
        t_r=1.0,
        memory='nilearn_cache',  # Cache computations
        memory_level=1,
        verbose=1
    )

    time_series_dict = {
        'active': [],
        'sham': []
    }

    for bold_file, confound_file in zip(bold_files, confounds_files):
        # Load confounds
        confounds = pd.read_csv(confound_file, sep='\t')

        # Select specific confound regressors
        selected_confounds = pd.concat([
            # Motion parameters and their derivatives/quadratic terms
            #confounds.filter(regex='^(trans|rot)_(x|y|z)($|_derivative1$|_power2$)'),
            confounds.filter(regex='^(trans|rot)_(x|y|z)($|_derivative1$)'),

            # CompCor components
            #confounds.filter(regex='^[at]_comp_cor_\d+'),

            # Global signals
            #confounds[['csf', 'white_matter', 'global_signal']],

            # Motion outliers
            #confounds.filter(regex='^motion_outlier'),

            # Edge/crown signals
            #confounds.filter(regex='^edge_')
        ], axis=1)

        # remove any columns in selected_confounds that have nans
        selected_confounds = selected_confounds.dropna(axis=1)

        # Extract time series with confound regression
        # This will now return time series for each probabilistic component
        time_series = masker.fit_transform(bold_file, confounds=selected_confounds)

        # Sort into conditions
        if 'ACTIVE' in bold_file:
            time_series_dict['active'].append(time_series)
        else:
            time_series_dict['sham'].append(time_series)

    return time_series_dict

In [20]:
def load_difumo_atlas(atlas_path):
    """
    Load DiFuMo atlas and verify its dimensions

    Parameters:
    -----------
    atlas_path : str
        Path to the DiFuMo atlas file

    Returns:
    --------
    atlas_img : Nifti1Image
        Loaded atlas image
    """
    from nilearn import image

    atlas_img = image.load_img(atlas_path)

    # Verify this is a 4D image with 1024 components
    if atlas_img.ndim != 4:
        raise ValueError("Expected 4D atlas image")
    if atlas_img.shape[-1] != 1024:
        raise ValueError(f"Expected 1024 components, got {atlas_img.shape[-1]}")

    return atlas_img

In [21]:
if generate_time_series:
    ## Generate a list of all available bold records from both ACTIVE and SHAM sessions
    
    # list all folders in data_root
    folders = [f for f in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, f))]
    bold_files = []
    confounds_files = []
    for folder in folders:
        # find file in output_folder that contains 'preproc_bold_resampled'
        output_folder = os.listdir(os.path.join(os.path.join(data_root,folder),'output'))
        for file in output_folder:
            if 'preproc_bold_resampled' in file:
                bold_files.append(os.path.join(os.path.join(data_root,folder),'output',file))
    
        # confounds timeseries
        input_folder=os.path.join(os.path.join(data_root,folder),'input')
        tmp=get_folders(os.path.join(os.path.join(data_root,folder),'input'))[0]
        tmp2=[x for x in get_folders(os.path.join(input_folder,tmp)) if 'sub-' in x]
        tmp3=os.path.join(os.path.join(input_folder,tmp),tmp2[0])
        tmp4=[os.path.join(tmp3,x) for x in os.listdir(tmp3) if 'ses-' in x][0]
        tmp5=[os.path.join(tmp4,x) for x in os.listdir(tmp4) if 'func' in x][0]
        confounds_files.append([os.path.join(tmp5,x) for x in os.listdir(tmp5) if 'confounds_timeseries.tsv' in x][0])
    
    print(f"Found {len(bold_files)} bold files")
    print(f"Found {len(confounds_files)} confounds files")

In [22]:
if generate_time_series:
    path_to_difumo = '/Users/jacekdmochowski/.cache/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_atlas-DiFuMo_desc-1024dimensions_probseg.nii.gz'
    try:
        difumo_atlas = load_difumo_atlas(path_to_difumo)
    except ValueError as e:
        print(f"Error loading DiFuMo atlas: {e}")
        print("Attempting to download from TemplateFlow")
        if path_to_difumo is None:
            path_to_difumo = api.get('MNI152NLin2009cAsym', atlas="DiFuMo", desc="1024dimensions", resolution=2, suffix="probseg", extension="nii.gz")
        difumo_atlas = nib.load(path_to_difumo)

In [23]:
if generate_time_series:
    time_series = load_and_prepare_data(bold_files, confounds_files, difumo_atlas)
    with open('../data/precomputed/difumo_time_series.pkl', 'wb') as f:
        pickle.dump(time_series, f)
else:
    with open('../data/precomputed/difumo_time_series.pkl', 'rb') as f:
        time_series = pickle.load(f)

## Let's take a look at the data, shall we ?!  Yeeaaahhh

In [25]:
print(time_series.keys())
print(type(time_series['active']),len(time_series['active']))
print(time_series['active'][0].shape) # 900 TRs, 1024 brain regions, yeaaaah

dict_keys(['active', 'sham'])
<class 'list'> 16
(900, 1024)


In [26]:
bold_3d_active = np.array(time_series['active'])
bold_3d_sham = np.array(time_series['sham'])
print(bold_3d_active.shape, bold_3d_sham.shape)

(16, 900, 1024) (16, 900, 1024)


## Initialize pandas dataframe that will store all functional connectivity values

In [27]:
df = pd.DataFrame(columns=['fc', 'roi1', 'roi2', 'subject', 'time_window', 'condition'])
# subject is a number from 0 to 15
# roi is a number from 0 to 1023
# only unique pairs of rois should be added (0,1), (0,2), ...  NOT (0,1) and (1,0)
# condition is either active or sham
# time_window is either pre, fus, or post
# fc is a floating point value

## Demonstrate how to compute functional connectivity for one row of the matrix

In [28]:
subject_idx = 0 # note: we never correlate between subjects
roi1 = 0 # zero based indexing!
roi2 = 1
time_segment = idx_pre

this_bold = bold_3d_active[subject_idx,time_segment,:] # the bold of this subject
corrmat = np.corrcoef(this_bold[:,[roi1,roi2]].T) # this is called a correlation matrix
fc = corrmat[0,1]

print(f"The functional connectivity between {labels[roi1][1]} and {labels[roi2][1]} for subject {subject_idx} during FUS is {fc:.3f}")

row = pd.DataFrame({'fc': [fc], 'roi1': [roi1], 'roi2': [roi2], 'subject': [subject_idx], 'time_window': ['pre'], 'condition': ['active']})

# add a row to df
df = pd.concat((df, row), axis=0)

The functional connectivity between Retrocalcarine cortex RH and Superior longitudinal fasciculus II mid-posterior RH for subject 0 during FUS is 0.393


/var/folders/g9/8jw7l6116kdd4pc549qnnz8r0000gn/T/ipykernel_20731/842304406.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df, row), axis=0)


In [29]:
df

,fc,roi1,roi2,subject,time_window,condition
0,0.392815,0,1,0,pre,active


Your task is to fill out this dataframe for all ROI pairs, subjects, conditions, and time windows